In [1]:
import igl
import math
import scipy as sp
import numpy as np
import meshplot as mp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from adaptmesh import triangulate
import pymesh
import numba
import os
root_folder = os.getcwd()

In [2]:
v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "prolate.off"))
### Parameters for running computation
global Kb
global Kv
global Ka
global gamma
global KbT
global delT
global Kal
Kal=1
gamma=2
Ka=2
Kv=1
Kb=0.01
H0=0
Volume_t= 0.95* 3.14 * 4 / 3
Area_t=4*3.14
KbT=0.01
charTimeStep=0.1
isAdaptiveStep=True
#hdt=0.5*dt
iterations = 2000
outfrequency = 100
tolerance = 1e-3
maxError = 2000
###Upsample_Here
#v_init,f=igl.upsample(v_init, f,1)
len(v_init)


472

In [3]:
###Energy Calculations /// Area+Bending+Volume
# @numba.jit
def Energy_area(v,f,Area_t):
    Area_new=cal_areatot(v,f)
    Energy_Area=Ka*((Area_new-Area_t)**2)/Area_t
    return Energy_Area
# @numba.jit
def Energy_volume(v,f,Volume_t):
    volume_new=cal_volumetot(v,f)
    Energy_volume=Kv*((volume_new-Volume_t)**2)/Volume_t
    return Energy_volume
# @numba.jit
# def Energy_bending(v,f,H0):
#     npv = igl.per_vertex_normals(v, f)
#     m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
#     minv = sp.sparse.diags(1 / m.diagonal())
#     area_voronoi=m.diagonal()
#     l = igl.cotmatrix(v, f) ###laplacian-operator
#     Hn = -minv.dot(l.dot(v))/2
#     H_mean = np.linalg.norm(Hn, axis=1)
#     sign_H = np.sign(np.sum(Hn*npv, axis=1))
#     H_mean_signed = H_mean*sign_H
#     #print(max(h_mean))
#     #print(min(h_mean))
#     Eb = 2*Kb*(((H_mean_signed-H0)**2)+K)*area_voronoi
#     total_EB = np.sum(Eb)
#     return total_EB
  

In [4]:
def adjacent_face(v,f):
    df=pd.DataFrame(f,columns=list('ABC'))
    row_numbers=[]
    for i in range(len(v)):
        row_numbers.append((df.index[(df['A'] == i)|(df['B'] == i) | (df['C'] == i)].tolist()))
    return row_numbers

def cal_areatot(v,f):
    dbl_area = igl.doublearea(v, f)    
    Areatot = np.sum(dbl_area)/2
    return Areatot
@numba.jit
def cal_volumetot(v,f):
    Volumetot = 0
    for i in range(len(f)):
        sum=0
        p0x=v[f[i][0]][0]
        p0y=v[f[i][0]][1]
        p0z=v[f[i][0]][2]
        p1x=v[f[i][1]][0]
        p1y=v[f[i][1]][1]
        p1z=v[f[i][1]][2]
        p2x=v[f[i][2]][0]
        p2y=v[f[i][2]][1]
        p2z=v[f[i][2]][2]
        v321= p2x*p1y*p0z
        v231= p1x*p2y*p0z
        v312= p2x*p0y*p1z
        v132= p0x*p2y*p1z
        v213= p1x*p0y*p2z
        v123= p0x*p1y*p2z
        sum=(-v321+ v231+ v312-v132-v213+ v123) / 6.0
        #print(sum)
        Volumetot+=sum
    return Volumetot

def areaGrad(v,f):
    #n=igl.per_vertex_normals(v,f) ## not using per_vertex_normals for areaGrad direction
    l = igl.cotmatrix(v, f) ###laplacian-operator
    ag = -l.dot(v)
    return ag
def volGrad(v,f):
    npv = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,npv)
    dbl_area = igl.doublearea(v, f)
    adjacent_vertices=igl.adjacency_list(f)
    adjacent_faces=adjacent_face(v,f)
    volumegrad=[]
    for i in range(len(v)):
        vol_ij=0
        for j in range(len(adjacent_faces[i])):
            k=adjacent_faces[i][j]
            Area=dbl_area[k]/2
            FaceNorm=face_normal[k]
            vol_ij += (1/3)*Area*FaceNorm
        volumegrad.append(vol_ij)
    return np.array(volumegrad)
def ver_new(v,f):
    adjacent_faces=adjacent_face(v,f)
    Area=igl.doublearea(v,f)
    n = igl.per_vertex_normals(v, f)
    face_normal=igl.per_face_normals(v,f,n)
    v_b=igl.barycenter(v,f)
    v_new=[]
    for i in range(len(v)):
        face_area=Area[adjacent_faces[i]]
        face_area_sum=np.sum(face_area)
        v_centroid=v_b[adjacent_faces[i]]
        sum_of_area_centroid=np.dot(face_area,v_centroid)
        v_avg=sum_of_area_centroid/face_area_sum
        fnorm=face_normal[adjacent_faces[i]]
        fsum=np.sum(fnorm,axis=0)
        lamda=(np.dot(v_avg,fsum)-np.dot(v[i],fsum))/np.dot(fsum,fsum)
        v_new.append(v_avg-lamda*fsum)    
    return np.array(v_new)

In [5]:
###Force  and Energy Calculations from bending
def Force_Bending(v,f):
    npv = igl.per_vertex_normals(v, f)
    K = igl.gaussian_curvature(v, f)
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    area_voronoi=m.diagonal()
    l = igl.cotmatrix(v, f) ###laplacian-operator
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    minv = sp.sparse.diags(1 / m.diagonal())
    Hn = -minv.dot(l.dot(v))/2
    H_mean = np.linalg.norm(Hn, axis=1)
    sign_H = np.sign(np.sum(Hn*npv, axis=1))
    #if (min(sign_H)<0):
    #    print('H_mean changes sign')
    H_mean_signed = H_mean*sign_H
    Lap_H = minv.dot(l.dot(H_mean_signed-H0))
    kn = minv.dot(K)
    first_term = 2*(H_mean_signed-H0)*(H_mean_signed**2 + H0*H_mean_signed -kn)
    totalforce = first_term + Lap_H


    #for i in range (len(v)):
    Force_bending = 2*Kb*npv*totalforce[:,None]*area_voronoi[:,None]
        #Force_Nodal.append(Force_density[i]*area_voronoi[i])
        #print(Force_density[i]) 
        #print(Force_Nodal[i])
    Eb = 2*Kb*(((H_mean_signed-H0)**2))*area_voronoi
    total_EB = np.sum(Eb)
        
    return Force_bending,total_EB

In [6]:
###Force from Area Constraints
def Force_Area(Area_t,grad_Area,Area_new):
    #grad_Area=volgrad(v,f)
    #Area_old=cal_volume(v,f)
    #Area_new=cal_Area(pos_new,f)
    Force_Area=-2*(Ka)*((Area_new-Area_t)/Area_t)*grad_Area
    return Force_Area

In [7]:
###Force from Volume Area Constraints
def Force_Volume(Volume_t,grad_Volume,volume_new):
    #grad_Volume=volgrad(v,f)
    #Volume_old=cal_volume(v,f)
    #Volume_new=cal_volume(pos_new,f)

    Force_Volume=-2*(Kv)*((volume_new-Volume_t)/Volume_t)*grad_Volume ## Volume constraint is not the same as in mem3dg
    return Force_Volume

In [8]:
def Total_Force(FB,FA,FV):
    Total_force=(FB+FA+FV)
    return Total_force

In [9]:
def updateTimeStep(v,f,l,TF,initialMaxForce,dt_size2_ratio,charTimeStep):
    currentMinSize = np.amin(l)
    currentMaxForce = np.amax(np.linalg.norm(TF, axis=1))
    dt = (dt_size2_ratio * currentMinSize **2)*\
        (initialMaxForce / currentMaxForce)
    
    if (charTimeStep / dt > 1e3):
        print("Time step too small! May consider restarting\n",
              "simulation in small time scale")
        print("Current size / initial size =",
              currentMinSize / sqrt(charTimeStep/dt_size2_ratio))
        print("Current forece / inital force =",
             currentMaxForce / initialMaxForce)
        exit()
    return dt

grad_A=areaGrad(v,f)

grad_Volume=volgrad(v,f)
grad_Volume=np.array(grad_Volume)
grad_Volume.shape

In [ ]:
%%time
# v_init, f = igl.read_triangle_mesh(os.path.join(root_folder, "Oblate_0.7.off"))### Can be changed for Prolate
l = igl.edge_lengths(v_init,f)
l0 = igl.avg_edge_length(v_init,f)
dt_size2_ratio = charTimeStep / np.amin(l)**2
number_of_vertices=len(v_init)
###Forward Euler Main 

#Setup

v=v_init
vel=np.zeros((len(v_init),3))
time=0
#FD=fun_FD2(v_init,f,vel)

#Volume_old=cal_volume(v,f)
#Area_t=cal_Area(v,f)
grad_Area=areaGrad(v_init,f)
grad_Volume=volGrad(v_init,f)
Area_new=cal_areatot(v_init,f)
print('initial area = ',Area_new)
Volume_new=cal_volumetot(v_init,f)
print('initial volume = ', Volume_new)

FB,EB=Force_Bending(v_init,f)
FA=Force_Area(Area_t,grad_Area,Area_new)
FV=Force_Volume(Volume_t,grad_Volume,Volume_new)
#FR=0*Force_Random(v,f,charTimeStep)
TF=Total_Force(FB,FA,FV)
initialMaxForce = np.amax(np.linalg.norm(TF, axis=1))

### Calculation of Energy
timeout=[]
totalEnergy=[]
EnergyArea=[]
EnergyVolume=[]
EnergyBending=[]
KineticEnergy=[]

for i in range(iterations):
    #Integration
    
    vel = TF/gamma
    
    ## adjust time step if adopt adaptive time step based on mesh size
    if (isAdaptiveStep):
        charTimeStep = updateTimeStep(v,f,l,TF,initialMaxForce,
                                                dt_size2_ratio,charTimeStep);
        
    dt = charTimeStep
    
    v += vel*dt
    time += dt
    
    l = igl.edge_lengths(v,f)
    v=ver_new(v,f)
    if (not igl.is_intrinsic_delaunay(l,f).all()):
        #v,f,_=pymesh.split_long_edges_raw(v,f,l0*4/3)
        print('time =',time,': not all edges are delaunay, call intrinsic delaunay triangulation')
        l, f= igl.intrinsic_delaunay_triangulation(l,f)
    
    v,f,_=pymesh.split_long_edges_raw(v,f,l0*4/3)
    v,f,__=pymesh.collapse_short_edges_raw(v,f, abs_threshold=0.0, rel_threshold=3/5, preserve_feature=True)

        #print('new vertice numbers=',len(v))
   
    #Force calculation
    Area_current=cal_areatot(v,f)
    grad_Area_new=areaGrad(v,f)
    Volume_current=cal_volumetot(v,f)
    grad_Volume_new=volGrad(v,f)

    ###Update forces here
 
    FB,EB=Force_Bending(v,f) ##bending_force
    FA=Force_Area(Area_t,grad_Area_new,Area_current)
    FV=Force_Volume(Volume_t,grad_Volume_new,Volume_current)

    TF=Total_Force(FB,FA,FV)

    
    mechErrorNorm = np.sum(np.linalg.norm(TF, axis=1))
    if (mechErrorNorm < tolerance) or (mechErrorNorm > maxError):
        print('time =',time)
        print('dt =',dt)
        print('dVolume/Volume_t =',Volume_current-Volume_t,'/',Volume_t)
        print('dArea/Area_t =',Area_current-Area_t,'/',Area_t)
        print('mechErrorNorm =',mechErrorNorm)
        timeout.append(i+1)
        EnergyArea.append(Energy_area(v,f,Area_t))
        EnergyVolume.append(Energy_volume(v,f,Volume_t))
        EnergyBending.append(EB)

        PotentialEnergy = np.array(EnergyArea) + np.array(EnergyVolume) + np.array(EnergyBending)
        print('PE = ',PotentialEnergy[-1])
        mp.jupyter()
        p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)
        name = 'test'+str(i)
        break
    if i%outfrequency==0:
        print('time =',time)
        print('dt =',dt)
        print('dVolume/Volume_t =',Volume_current-Volume_t,'/',Volume_t)
        print('dArea/Area_t =',Area_current-Area_t,'/',Area_t)
        print('mechErrorNorm =',mechErrorNorm)
        timeout.append(time)
        EnergyArea.append(Energy_area(v,f,Area_t))
        EnergyVolume.append(Energy_volume(v,f,Volume_t))
        EnergyBending.append(EB)
        PotentialEnergy=np.array(EnergyArea) + np.array(EnergyVolume) + np.array(EnergyBending)
        
        print('PE = ',PotentialEnergy[-1])

        mp.jupyter()
        p=mp.plot(v,f,shading={"wireframe":True, "wire_color": "black", #Wireframerendering
                               "width": 300, "height": 300},return_plot=True)
        #p.add_lines(v,v+1*FB, shading={"line_color": "red"})
        name = 'test'+str(i)
        print('number of vertices=',len(v))
        print('number of Iterations=',i)
    if i>=100 and i%outfrequency==0:
        EnergyChangeRate=(abs(PotentialEnergy[-1]-PotentialEnergy[-2])/(PotentialEnergy[-1]))*100
        print('Energy_Change_Rate =', EnergyChangeRate)
    if i>=100 and EnergyChangeRate<tolerance:
        print('Change of Energy is very small \n Reached Equilibrioum Shape')
        break


initial area =  12.477707688493407
initial volume =  3.809031960721252
time = 0.1
dt = 0.1
dVolume/Volume_t = -0.16596306757624912 / 3.9773333333333336
dArea/Area_t = -0.0780029596444276 / 12.56
mechErrorNorm = 1.8351493071945082
PE =  0.29733866596087805


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001232…

number of vertices= 472
number of Iterations= 0
time = 12.70434195479261
dt = 0.14177636101497607
dVolume/Volume_t = -0.07996935620650047 / 3.9773333333333336
dArea/Area_t = 0.05446459025108652 / 12.56
mechErrorNorm = 0.4629347784823319
PE =  0.28690132922939054


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0002767…

number of vertices= 472
number of Iterations= 100
Energy_Change_Rate = 3.63795342444802
time = 29.646378974314366
dt = 0.1986532330165664
dVolume/Volume_t = -0.05865934148280472 / 3.9773333333333336
dArea/Area_t = 0.04359515290582827 / 12.56
mechErrorNorm = 0.3862114914747115
PE =  0.2822302653925379


...

number of vertices= 472
number of Iterations= 1000
Energy_Change_Rate = 0.3504163713761086
time = 266.3697417365192
dt = 0.3269905565484528
dVolume/Volume_t = 0.06333619080215547 / 3.9773333333333336
dArea/Area_t = -0.04722025630021953 / 12.56
mechErrorNorm = 0.1464691042139673
PE =  0.26360404405546783


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000411…

number of vertices= 472
number of Iterations= 1100
Energy_Change_Rate = 0.29988924070088063
time = 292.2869478821393
dt = 0.33528111686127265
dVolume/Volume_t = 0.06966504664583129 / 3.9773333333333336
dArea/Area_t = -0.052186927203331734 / 12.56
mechErrorNorm = 0.1368014890960802
PE =  0.2629227823857627


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000454…

number of vertices= 472
number of Iterations= 1200
Energy_Change_Rate = 0.25911093117277295


In [ ]:
time=np.linspace(0,iterations,len(EnergyBending))
# plt.plot(time,totalEnergy,'o-', color='red', label='total Energy')
plt.plot(time,EnergyBending,'o-', label='Bending Energy')
plt.plot(time,EnergyVolume,'v-', label='Volume Energy')
plt.plot(time,EnergyArea,'o-', label='Area Energy')
plt.plot(time,PotentialEnergy,'x-',label='Potential Energy')
# plt.plot(time,KineticEnergy,'v-', label='Kinetic Energy',color='k')
#plt.axis([0, 800, 0, 2])
plt.legend(loc='right')
plt.title('timesteps vs Energy')
plt.show()

In [ ]:
PotentialEnergy

In [ ]:
EnergyBending